In [1]:
import numpy as np 
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
# sklean接口的包装器KerasClassifier，作为sklearn的分类器接口
from keras.wrappers.scikit_learn import KerasClassifier
# 穷搜所有特定的参数值选出最好的模型参数
from sklearn.grid_search import GridSearchCV

Using TensorFlow backend.
D:\anaconda1\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\anaconda1\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# 类别的数目
nb_classes = 10
# 输入图像的维度
img_rows, img_cols = 28, 28
# 读取数据
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 读取的数据不包含通道维，因此shape为(60000,28,28)
# 为了保持和后端tensorflow的数据格式一致，将数据补上通道维
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
# 新的数据shape为 (60000,28,28,1)， 1代表通道是1，也就是灰阶图片
# 指明输入数据的大小，便于后面搭建网络的第一层传入该参数
input_shape = (img_rows, img_cols, 1)
# 数据类型改为float32，单精度浮点数
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# 数据归一化（图像数据常用）
X_train /= 255
X_test /= 255
# 将类别标签转换为one-hot编码
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [3]:
# 定义配置卷积网络模型的函数
def make_model(dense_layer_sizes, nb_filters, nb_conv, nb_pool):
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                            border_mode='valid',
                            input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return model 

In [4]:
# 全连接层的备选参数列表
dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
# 实现为Keras准备的sklearn分类器接口，创建一个分类器/评估器对象
# 传入的参数为：
# build_fn: callable function or class instance
# **sk_params: model parameters & fitting parameters
# 具体分析如下：
# 传入的第一个参数(build_fn)为可回调的函数，该函数建立、配置并返回一个Keras model，
# 该model将被用来训练/预测，这里我们传入了刚刚定义好的make_model函数
# 传入的第二个参数(**sk_params)为关键字参数(关键字参数在函数内部自动组装为一个dict),
# 既可以是模型的参数，也可以是训练的参数，合法的模型参数就是build_fn的参数，
# 注意，像所有sklearn中其他的评估器(estimator)一样，build_fn应当为其参数提供默认值，
# 以便我们在建立estimator的时候不用向sk_params传入任何值。
# sk_params也可以接收用来调用fit/predict/predict_proba/score方法的参数，
# 例如'nb_epoch','batch_size'
# fit/predict/predict_proba/score方法的参数将会优先从传入fit/predict/predict_proba/score
# 的字典参数中选择，其次才从传入sk_params的参数中选，最后才选择keras的Sequential模型的默认参数中选择
# 这里我们传入了用于调用fit方法的batch_size参数
my_classifier = KerasClassifier(make_model, batch_size=32)
# 当调用sklearn的grid_search接口时，合法的可调参数就是传给sk_params的参数，包括训练参数
# 换句话说，就是可以用grid_search来选择最佳的batch_size/nb_epoch，或者其他的一些模型参数

# GridSearchCV类，穷搜(Exhaustive search)评估器中所有特定的参数，
# 其重要的两类方法为fit和predict
# 传入参数为评估器对象my_classifier，由每一个grid point实例化一个estimator
# 参数网格param_grid，类型为dict，需要尝试的参数名称以及对应的数值
# 评估方式scoring，这里采用对数损失来评估
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     'nb_epoch': [3,6],
                                     'nb_filters': [8],
                                     'nb_conv': [3],
                                     'nb_pool': [2]},
                         scoring='log_loss')
# 根据各个参数值的不同组合在(X_train, y_train)上训练模型
validator.fit(X_train, y_train)
# 打印出训练过程中找到的最佳参数
print('Yhe parameters of the best model are: ')
print(validator.best_params_)

Epoch 1/3
40000/40000 [==============================] - 19s - loss: 0.8695 - acc: 0.7072    
Epoch 2/3
40000/40000 [==============================] - 20s - loss: 0.5314 - acc: 0.8258    
Epoch 3/3
  928/20000 [>.............................] - ETA: 4s

D:\anaconda1\lib\site-packages\sklearn\metrics\scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


20000/20000 [==============================] - 3s     
Epoch 1/3
40000/40000 [==============================] - 20s - loss: 0.7154 - acc: 0.7692    
Epoch 2/3
40000/40000 [==============================] - 20s - loss: 0.4404 - acc: 0.8626    
Epoch 3/3
  864/20000 [>.............................] - ETA: 4s

D:\anaconda1\lib\site-packages\sklearn\metrics\scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


20000/20000 [==============================] - 3s     
Epoch 1/3
40000/40000 [==============================] - 19s - loss: 0.8714 - acc: 0.7119    
Epoch 2/3
40000/40000 [==============================] - 19s - loss: 0.5244 - acc: 0.8347    
Epoch 3/3
  800/20000 [>.............................] - ETA: 4s

...

40000/40000 [==============================] - 23s - loss: 0.6006 - acc: 0.8088    
Epoch 2/6
40000/40000 [==============================] - 22s - loss: 0.3099 - acc: 0.9077    
Epoch 3/6
40000/40000 [==============================] - 21s - loss: 0.2356 - acc: 0.9313    
Epoch 4/6
40000/40000 [==============================] - 20s - loss: 0.2070 - acc: 0.9395    
Epoch 5/6
40000/40000 [==============================] - 21s - loss: 0.1822 - acc: 0.9473    
Epoch 6/6
40000/40000 [==============================] - 21s - loss: 0.1617 - acc: 0.9537    


D:\anaconda1\lib\site-packages\sklearn\metrics\scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


40000/40000 [==============================] - 21s - loss: 0.5787 - acc: 0.8175    
Epoch 2/6
40000/40000 [==============================] - 23s - loss: 0.2871 - acc: 0.9145    
Epoch 3/6
40000/40000 [==============================] - 20s - loss: 0.2309 - acc: 0.9327    
Epoch 4/6
40000/40000 [==============================] - 21s - loss: 0.1983 - acc: 0.9423    
Epoch 5/6
40000/40000 [==============================] - 20s - loss: 0.1732 - acc: 0.9502    
Epoch 6/6
40000/40000 [==============================] - 20s - loss: 0.1541 - acc: 0.9549    


D:\anaconda1\lib\site-packages\sklearn\metrics\scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


60000/60000 [==============================] - 31s - loss: 0.4721 - acc: 0.8531    
Epoch 2/6
60000/60000 [==============================] - 30s - loss: 0.2281 - acc: 0.9340    
Epoch 3/6
60000/60000 [==============================] - 31s - loss: 0.1836 - acc: 0.9482    
Epoch 4/6
60000/60000 [==============================] - 30s - loss: 0.1530 - acc: 0.9555    
Epoch 5/6
60000/60000 [==============================] - 35s - loss: 0.1420 - acc: 0.9593    
Epoch 6/6
60000/60000 [==============================] - 32s - loss: 0.1309 - acc: 0.9634    
Yhe parameters of the best model are: 
{'dense_layer_sizes': [64, 64], 'nb_conv': 3, 'nb_pool': 2, 'nb_filters': 8, 'nb_epoch': 6}


## 利用keras与sklearn进行分类和回归

In [ ]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:, 0:13]
Y = dataset[:, 13]
# define base mode
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=13, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, nb_epoch=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
# use 10-fold cross validation to evaluate this baseline model
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))